__Run basic contamination screening and fastqc. perform mapping and alignment, following this tutorial https://www.biostars.org/p/41951/__

In [26]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/2022-02-07-fastq_screen/


/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/2022-02-07-fastq_screen


In [ ]:
#make a list of read filenames to use for next loops
!ls *R2.fastq.gz > filenames
!sed -i'' -e 's/R2.fastq.gz//g' filenames #remove the extension

In [ ]:
#run fastqc 
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/2022-02-07-fastq_screen/*.fastq.gz; do fastqc -t 34 *.fastq.gz $i; done
#visualize sequencing quality for all samples in trimmed_reads directory. check that all barcode sequences are removed
!multiqc .

Started analysis of CAP10_APPC_34.05_R1.fastq.gz
Approx 5% complete for CAP10_APPC_34.05_R1.fastq.gz
Started analysis of CAP10_APPC_34.05_R2.fastq.gz
Approx 10% complete for CAP10_APPC_34.05_R1.fastq.gz
Approx 5% complete for CAP10_APPC_34.05_R2.fastq.gz
Started analysis of CAP10_APPC_34.06_R1.fastq.gz
Approx 15% complete for CAP10_APPC_34.05_R1.fastq.gz
Approx 10% complete for CAP10_APPC_34.05_R2.fastq.gz
Approx 5% complete for CAP10_APPC_34.06_R1.fastq.gz
Approx 20% complete for CAP10_APPC_34.05_R1.fastq.gz
Started analysis of CAP10_APPC_34.06_R2.fastq.gz
Approx 15% complete for CAP10_APPC_34.05_R2.fastq.gz
Approx 25% complete for CAP10_APPC_34.05_R1.fastq.gz
Approx 10% complete for CAP10_APPC_34.06_R1.fastq.gz
Approx 5% complete for CAP10_APPC_34.06_R2.fastq.gz
Approx 20% complete for CAP10_APPC_34.05_R2.fastq.gz
Started analysis of CAP10_APPC_34.13_R1.fastq.gz
Approx 30% complete for CAP10_APPC_34.05_R1.fastq.gz
Started analysis of CAP10_APPC_34.13_R2.fastq.gz
Approx 15% complete f

Started analysis of CAP11_APPC_37.10_R2.fastq.gz
Approx 95% complete for CAP10_APPC_34.06_R2.fastq.gz
Approx 20% complete for CAP10_APPC_36.06_R1.fastq.gz
Approx 90% complete for CAP10_APPC_34.06_R1.fastq.gz
Approx 30% complete for CAP10_APPC_35.11_R1.fastq.gz
Approx 65% complete for CAP10_APPC_35.07_R2.fastq.gz
Approx 25% complete for CAP10_APPC_36.04_R2.fastq.gz
Approx 35% complete for CAP10_APPC_36.04_R1.fastq.gz
Analysis complete for CAP10_APPC_34.05_R2.fastq.gz
Approx 80% complete for CAP10_APPC_35.07_R1.fastq.gz
Approx 10% complete for CAP10_APPC_35.14_R2.fastq.gz
Approx 5% complete for CAP11_APPC_37.10_R1.fastq.gz
Approx 15% complete for CAP10_APPC_36.06_R2.fastq.gz
Approx 25% complete for CAP10_APPC_36.06_R1.fastq.gz
Approx 40% complete for CAP10_APPC_36.04_R1.fastq.gz
Started analysis of CAP11_APPC_37.12_R1.fastq.gz
Approx 25% complete for CAP10_APPC_35.11_R2.fastq.gz
Approx 70% complete for CAP10_APPC_35.07_R2.fastq.gz
Approx 95% complete for CAP10_APPC_34.06_R1.fastq.gz
Anal

Analysis complete for CAP10_APPC_36.06_R1.fastq.gz
Approx 15% complete for CAP11_APPC_38.12_R2.fastq.gz
Started analysis of CAP11_APPC_39.15_R1.fastq.gz
Approx 5% complete for CAP11_APPC_39.12_R1.fastq.gz
Approx 50% complete for CAP11_APPC_37.10_R1.fastq.gz
Approx 70% complete for CAP10_APPC_35.11_R1.fastq.gz
Approx 60% complete for CAP10_APPC_35.11_R2.fastq.gz
Approx 85% complete for CAP10_APPC_36.06_R2.fastq.gz
Approx 5% complete for CAP11_APPC_39.12_R2.fastq.gz
Approx 30% complete for CAP10_APPC_34.14_R1.fastq.gz
Approx 30% complete for CAP10_APPC_35.14_R1.fastq.gz
Approx 15% complete for CAP11_APPC_37.15_R1.fastq.gz
Approx 45% complete for CAP11_APPC_37.10_R2.fastq.gz
Approx 20% complete for CAP11_APPC_38.12_R1.fastq.gz
Approx 10% complete for CAP11_APPC_38.14_R1.fastq.gz
Started analysis of CAP11_APPC_39.15_R2.fastq.gz
Approx 90% complete for CAP10_APPC_36.06_R2.fastq.gz
Approx 25% complete for CAP10_APPC_34.13_R1.fastq.gz
Approx 55% complete for CAP11_APPC_37.10_R1.fastq.gz
Start

Approx 20% complete for CAP12_APPC_40.15_R1.fastq.gz
Approx 55% complete for CAP12_APPC_40.09_R1.fastq.gz
Approx 45% complete for CAP11_APPC_38.12_R2.fastq.gz
Approx 60% complete for CAP12_APPC_40.09_R2.fastq.gz
Approx 25% complete for CAP11_APPC_38.14_R2.fastq.gz
Approx 35% complete for CAP11_APPC_37.15_R1.fastq.gz
Started analysis of CAP12_APPC_41.09_R1.fastq.gz
Approx 10% complete for CAP11_APPC_37.12_R1.fastq.gz
Approx 50% complete for CAP11_APPC_38.12_R1.fastq.gz
Approx 35% complete for CAP11_APPC_39.12_R1.fastq.gz
Approx 40% complete for CAP10_APPC_34.14_R2.fastq.gz
Approx 30% complete for CAP11_APPC_39.12_R2.fastq.gz
Approx 35% complete for CAP10_APPC_34.13_R2.fastq.gz
Approx 60% complete for CAP12_APPC_40.09_R1.fastq.gz
Approx 15% complete for CAP12_APPC_40.15_R2.fastq.gz
Approx 65% complete for CAP12_APPC_40.09_R2.fastq.gz
Approx 95% complete for CAP11_APPC_37.10_R2.fastq.gz
Approx 30% complete for CAP11_APPC_39.03_R2.fastq.gz
Approx 35% complete for CAP11_APPC_37.15_R2.fastq.

Approx 45% complete for CAP11_APPC_38.14_R2.fastq.gz
Approx 85% complete for CAP11_APPC_38.12_R1.fastq.gz
Approx 55% complete for CAP10_APPC_34.14_R2.fastq.gz
Approx 25% complete for CAP11_APPC_39.14_R1.fastq.gz
Approx 60% complete for CAP11_APPC_39.12_R2.fastq.gz
Approx 60% complete for CAP10_APPC_34.14_R1.fastq.gz
Approx 20% complete for CAP12_APPC_41.10_R2.fastq.gz
Approx 65% complete for CAP11_APPC_39.12_R1.fastq.gz
Approx 20% complete for CAP11_APPC_40.11_R1.fastq.gz
Approx 55% complete for CAP11_APPC_39.03_R2.fastq.gz
Approx 70% complete for CAP10_APPC_35.14_R2.fastq.gz
Approx 15% complete for CAP10_APPC_35.02_R1.fastq.gz
Approx 50% complete for CAP11_APPC_39.15_R1.fastq.gz
Approx 55% complete for CAP12_APPC_40.15_R2.fastq.gz
Approx 25% complete for CAP11_APPC_39.14_R2.fastq.gz
Approx 5% complete for CAP12_APPC_42.06_R2.fastq.gz
Approx 60% complete for CAP12_APPC_40.15_R1.fastq.gz
Approx 20% complete for CAP11_APPC_40.11_R2.fastq.gz
Approx 85% complete for CAP11_APPC_38.12_R2.fas

...

Approx 20% complete for CAP3_APPC_11.19_R1.fastq.gz
Approx 45% complete for CAP3_APPC_11.16_R2.fastq.gz
Approx 60% complete for CAP3_APPC_11.16_R1.fastq.gz
Approx 35% complete for CAP2_APPC_10.15_R2.fastq.gz
Approx 75% complete for CAP3_APPC_11.10_R1.fastq.gz
Approx 65% complete for CAP3_APPC_11.10_R2.fastq.gz
Started analysis of CAP3_APPC_12.18_R1.fastq.gz
Approx 15% complete for CAP2_APPC_10.10_R1.fastq.gz
Approx 60% complete for CAP2_APPC_40.06_R2.fastq.gz
Approx 75% complete for CAP2_APPC_40.06_R1.fastq.gz
Approx 65% complete for CAP3_APPC_11.16_R1.fastq.gz
Approx 15% complete for CAP3_APPC_11.27_R1.fastq.gz
Approx 15% complete for CAP3_APPC_11.19_R2.fastq.gz
Approx 10% complete for CAP3_APPC_11.27_R2.fastq.gz
Approx 50% complete for CAP3_APPC_11.16_R2.fastq.gz
Approx 80% complete for CAP3_APPC_11.10_R1.fastq.gz
Approx 85% complete for CAP1_APPC_21.01_R1.fastq.gz
Approx 70% complete for CAP3_APPC_11.10_R2.fastq.gz
Approx 25% complete for CAP3_APPC_11.19_R1.fastq.gz
Approx 70% compl

Approx 5% complete for CAP3_APPC_12.19_R2.fastq.gz
Approx 80% complete for CAP3_APPC_11.27_R1.fastq.gz
Approx 85% complete for CAP3_APPC_12.19_R1.fastq.gz
Approx 10% complete for CAP3_APPC_12.19_R2.fastq.gz
Approx 15% complete for CAP3_APPC_12.19_R2.fastq.gz
Approx 90% complete for CAP3_APPC_12.19_R1.fastq.gz
Approx 45% complete for CAP2_APPC_10.15_R2.fastq.gz
Approx 20% complete for CAP3_APPC_12.19_R2.fastq.gz
Approx 95% complete for CAP3_APPC_12.19_R1.fastq.gz
Approx 80% complete for CAP15_APPC_29.01_R1.fastq.gz
Approx 25% complete for CAP3_APPC_12.19_R2.fastq.gz
Analysis complete for CAP3_APPC_12.19_R1.fastq.gz
Approx 30% complete for CAP3_APPC_12.19_R2.fastq.gz
Approx 60% complete for CAP3_APPC_11.19_R2.fastq.gz
Approx 35% complete for CAP3_APPC_12.19_R2.fastq.gz
Approx 70% complete for CAP3_APPC_11.19_R1.fastq.gz
Approx 40% complete for CAP3_APPC_12.19_R2.fastq.gz
Approx 80% complete for CAP3_APPC_11.27_R2.fastq.gz
Approx 45% complete for CAP3_APPC_12.19_R2.fastq.gz
Approx 90% com

Approx 60% complete for CAP3_APPC_12.22_R2.fastq.gz
Approx 85% complete for CAP15_APPC_29.01_R1.fastq.gz
Approx 25% complete for CAP3_APPC_13.18_R2.fastq.gz
Approx 50% complete for CAP3_APPC_12.24_R1.fastq.gz
Started analysis of CAP4_APPC_13.29_R1.fastq.gz
Approx 75% complete for CAP3_APPC_12.22_R1.fastq.gz
Approx 90% complete for CAP1_APPC_28.17_R2.fastq.gz
Approx 40% complete for CAP3_APPC_12.24_R2.fastq.gz
Approx 60% complete for CAP3_APPC_12.18_R2.fastq.gz
Approx 65% complete for CAP2_APPC_10.15_R1.fastq.gz
Approx 15% complete for CAP3_APPC_13.25_R2.fastq.gz
Analysis complete for CAP1_APPC_28.17_R1.fastq.gz
Approx 65% complete for CAP3_APPC_12.22_R2.fastq.gz
Approx 75% complete for CAP3_APPC_12.18_R1.fastq.gz
Approx 30% complete for CAP3_APPC_13.18_R1.fastq.gz
Approx 80% complete for CAP3_APPC_12.22_R1.fastq.gz
Approx 45% complete for CAP3_APPC_12.24_R2.fastq.gz
Approx 30% complete for CAP3_APPC_13.18_R2.fastq.gz
Approx 55% complete for CAP3_APPC_12.24_R1.fastq.gz
Approx 20% comple

Approx 70% complete for CAP3_APPC_13.25_R1.fastq.gz
Approx 95% complete for CAP3_APPC_13.18_R1.fastq.gz
Approx 15% complete for CAP4_APPC_14.26_R2.fastq.gz
Approx 25% complete for CAP4_APPC_14.26_R1.fastq.gz
Approx 25% complete for CAP2_APPC_10.10_R1.fastq.gz
Approx 95% complete for CAP3_APPC_13.18_R2.fastq.gz
Approx 25% complete for CAP4_APPC_13.29_R2.fastq.gz
Started analysis of CAP4_APPC_14.29_R1.fastq.gz
Approx 25% complete for CAP2_APPC_10.10_R2.fastq.gz
Approx 60% complete for CAP13_APPC_05.08_R1.fastq.gz
Approx 20% complete for CAP4_APPC_14.26_R2.fastq.gz
Approx 70% complete for CAP2_APPC_10.15_R2.fastq.gz
Approx 30% complete for CAP4_APPC_14.26_R1.fastq.gz
Approx 10% complete for CAP4_APPC_14.23_R1.fastq.gz
Approx 70% complete for CAP3_APPC_13.25_R2.fastq.gz
Approx 5% complete for CAP4_APPC_14.23_R2.fastq.gz
Approx 75% complete for CAP3_APPC_13.25_R1.fastq.gz
Analysis complete for CAP3_APPC_13.18_R1.fastq.gz
Analysis complete for CAP3_APPC_13.18_R2.fastq.gz
Approx 25% complete 

Approx 5% complete for CAP5_APPC_17.09_R2.fastq.gz
Approx 30% complete for CAP4_APPC_16.15_R2.fastq.gz
Approx 45% complete for CAP4_APPC_16.14_R2.fastq.gz
Approx 90% complete for CAP2_APPC_10.15_R2.fastq.gz
Approx 35% complete for CAP4_APPC_14.09_R1.fastq.gz
Approx 30% complete for CAP4_APPC_16.15_R1.fastq.gz
Analysis complete for CAP2_APPC_10.15_R1.fastq.gz
Approx 75% complete for CAP4_APPC_13.29_R1.fastq.gz


In [ ]:
#for some reason, barcodes still in at the start of read2, so use cutadapt to remove the first 6 + reads from the read2 files 5', and 1 bp 3'. read 1 files remove 1 bp 5' and 1 bp 3'. run in the python2.7 environment
!rename '_R' '.R' *.fastq.gz 
for i in $(cat filenames); do cutadapt -u 1 -u -1 -U 7 -U -1 -o $i\.R1.cut.fastq.gz -p $i\.R2.cut.fastq.gz $i\.R1.fastq.gz $i\.R2.fastq.gz; done

In [ ]:
cutadapt -a ADAPTER_FWD -A ADAPTER_REV -o out.1.fastq -p out.2.fastq reads.1.fastq reads.2.fastq

In [ ]:
#screen for contamination and filter out tagged reads simultaneously, filter 1 means keep only reads that mapped exclusively to the reference for A. percula. since this is already done for the read 1 files, just need to do it for the read 2 files that I cut the barcode from  
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*.cut.fastq.gz; do fastq_screen --tag --filter 1 --threads 32 --outdir /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/ $i; done

In [ ]:
#run fastqc 
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*tagged_filter.fastq.gz; do fastqc -t 6 *tagged_filter.fastq.gz  $i; done

In [ ]:
#visualize sequencing quality for all samples in trimmed_reads directory. check that all barcode sequences are removed
!multiqc .

In [ ]:
#so the read 1 and read 2 files still fail qc because the A/T barcode overhang hookup, trim 1 bp on either side of fragments for all read files, check again and then map
!ls *.tagged_filter.fastq.gz  > filenames
!sed -i'' -e 's/.fastq.gz//g' filenames #remove the extension
for i in $(cat filenames); do cutadapt -u 1 -u -1 -o $i.cut.fastq.gz $i.fastq.gz; done
#delete the unfilered fastas from this directory, since they are still in ultraplex out
#remove cut from filenames so names are consistent
!rename '.cut' '' *.fastq.gz 
!rename '.cut' '' *.fastq.gz #second time because the read 2 files have two cut name spaces because they were cut twice
#run fastqc and multiqc again to check we are ready to map
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*tagged_filter.fastq.gz; do fastqc -t 30 *tagged_filter.fastq.gz  $i; done
#make sure to push the multiqc report to github

In [30]:
!zcat CAP1_APPC_16.11.R1.fastq.gz | head
!zcat CAP1_APPC_16.11.R2.fastq.gz | head



@A00741:178:H55NJDRXY:2:2101:12816:10161:N:0:0rbc:
TTGATCTCTGCTCTGCTGGTGTCAGGTTTCACTTTGTTCTAACAGCTCCTTATAT
+
,FF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFF
@A00741:178:H55NJDRXY:2:2101:7365:10311:N:0:0rbc:
ACTCTGTTGTTTTGGCTCTAATAACATGAATATTAAACTGAGCGGAGTGACGA
+
FFFFF,::FFF,FFFFFFFFFFFF,FF,FFFFFFFFF:FFFFFFFFF,FFFFF
@A00741:178:H55NJDRXY:2:2101:18819:10471:N:0:0rbc:
ACCTTCTTACCTCAGTAAGAAATGAACCGCTGTCATAGCCCTAGTATTGCTAATC

gzip: stdout: Broken pipe
@A00741:178:H55NJDRXY:2:2101:12816:10164:N:0:0rbc:
ATCACGAATCTGAGGTTCTTCAAATAAATTCAGGTTCCTGTCAAAAGGAACCAGGAAATAT
+
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFF:FFFFFFFF
@A00741:178:H55NJDRXY:2:2101:7365:10314:N:0:0rbc:
ATCACGATAAACATATTTTATCTCTAATTAACCTACAACTAGCACCAAAATTACACTTTTT
+
FFFF:F:FFFFFFFFFFFF,FFFFFFFFFFFFFFFFFF:FFF:F,:FFFF:FFFFFFF,FF
@A00741:178:H55NJDRXY:2:2101:18819:10474:N:0:0rbc:
ATCACGAAAGCATTGAATGAGAAGGTGTGTCCAAACTTTTGACTGGTAGTGTATTTCATCC

gzip: stdout: Broken pipe


In [31]:
#with the previous pipeline
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/Pop01_APPC1611.R1.fq.gz | head
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/Pop01_APPC1611.R2.fq.gz | head


@2_2101_24885_8891/1
GGTAGTTCATGCGTAGCCATACCATTAGCCTATTGTTGTGTTGTTAGTGCCTTT
+
FFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFF:,FFFFFFFFFFFFFFFFFF
@2_2101_32353_8891/1
CTTTGGTCATTTTCACCTCAACTGGAATCCTTTGGTTTCTGTCCACAATCTTCT
+
FFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFF
@2_2101_3721_8907/1
GGTTAACAGATGCCTTAAAACTTGTGACCATTAGTCTTGATTTCACATTTAGAC

gzip: stdout: Broken pipe
@2_2101_24885_8891/2
TCACGTACTAACAACACAGCTAACTATGAAGCTAATGACCACTAAAGACACAGTTATTGC
+
FFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
@2_2101_32353_8891/2
TCACGAGGTCTCCCCAAAATCCTGACTTAAACCCATCAAGAACCTGTGGACTGATGAAGA
+
FFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
@2_2101_3721_8907/2
TCACGATTAGACTTTAGAAGATCAAATTAAACAGAAGAGACAATAAAACGATCAAAAAGA

gzip: stdout: Broken pipe


In [36]:
#previous pipeline, earlier step
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/Pop01_APPC1611.F.fq.gz | head
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/Pop01_APPC1611.R.fq.gz | head

@2_2101_12816_1016/1
TGATCTCTGCTCTGCTGGTGTCAGGTTTCACTTTGTTCTAACAGCTCCTTATAT
+
FF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFF
@2_2101_7365_1031/1
CTCTGTTGTTTTGGCTCTAATAACATGAATATTAAACTGAGCGGAGTGACGAAG
+
FFFF,::FFF,FFFFFFFFFFFF,FF,FFFFFFFFF:FFFFFFFFF,FFFFFFF
@2_2101_28989_1031/1
CAACCAGTCAAAGGTTATTACTGAGGATTTTAAACAAAAACATTCAGGCAAACA

gzip: stdout: Broken pipe
@2_2101_12816_1016/2
TCACGAATCTGAGGTTCTTCAAATAAATTCAGGTTCCTGTCAAAAGGAACCAGGAAATAT
+
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFF:FFFFFFFF
@2_2101_7365_1031/2
TCACGATAAACATATTTTATCTCTAATTAACCTACAACTAGCACCAAAATTACACTTTTT
+
FFF:F:FFFFFFFFFFFF,FFFFFFFFFFFFFFFFFF:FFF:F,:FFFF:FFFFFFF,FF
@2_2101_28989_1031/2
TCACGTGATTACAAAAAAAATCTGTATCGACCCGGAAAAAAAACAAACATGTCAGGCACC

gzip: stdout: Broken pipe


In [34]:
#omg last try- unzip all fastas and try fastq_pair
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/2022-02-07-fastq_screen/repair/
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/2022-02-07-fastq_screen/*.fastq.gz; do gunzip $i; done

/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/2022-02-07-fastq_screen/repair
^C


In [ ]:
#(commandline) [katrinac@annotate DNA]$ samtools stats filter.merged.bam > samtools_stats
#looks like only 22% reads properly paired- go back to trim the barcodes with cutadapt but definitely in paired end mode!!

In [ ]:
#go back to previous piepline files with correct read names, use cutadapt on these
#something like 
for i in $(cat cutadapt_totrim7); do cutadapt -u 1 -u -1 -U 7 -U -1 -o $i\.R1.cut.fastq.gz -p $i\.R2.cut.fastq.gz $i\.R1.fastq.gz $i\.R2.fastq.gz; done

In [ ]:
#okay, looks like still weird artifacts of library prep, not barcodes, that are present in 68 read files up to position 7 bp on 5' end. move all filtered files to barcodes_cut directory and use cutadapt once more
for i in $(cat reads_to_cut7); do cutadapt -u 7 -o $i.cut.fastq.gz $i.fastq.gz; done
#rename cut files, remove uncut files
#fastqc one more time, multiqc to check, then map

In [ ]:
#for some reason CAP10_APPC_34.13_R1.tagged_filter.fastq.gz was not including in multiqc report, use fastqc to check if it's good to map with
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/barcodes_cut/

!fastqc -t 30 CAP10_APPC_34.13_R1.tagged_filter.fastq.gz
#yes looks fine, can move on to mapping

In [ ]:
#collate the names of samples that were in the final cutting round of first 7 bp, and the files that were fine and move into a mapping directory
#!mkdir mapping
!for i in $(cat clean_reads); do mv $i mapping/; done
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/barcodes_cut/mapping


In [ ]:
#make bwa index for reference fasta to use in mapping
!bwa index /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/reference.fasta


In [ ]:
#remove the underscore after the sample names of files to map because it was a problem in cutadapt loop
!rename '.cut' '' *.fastq.gz 
!ls  *R1.tagged_filter.fastq.gz > filenames
!sed -i'' -e 's/_R1.tagged_filter.fastq.gz//g' filenames #remove the extension

In [ ]:
#map using default parameters but add -M for picard compatibility, make sure to use the tagged/filtered
!for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/barcodes_cut/mapping/filenames); do bwa mem -t 34 -M -R "@RG\tID:$i\tSM:$i\tPL:Illumina" /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/reference.fasta $i\_R1.tagged_filter.fastq.gz $i\_R2.tagged_filter.fastq.gz  2> bwa.$i.log | samtools sort -@ 34 -o $i-RG.bam 2>$i.bam.log; done  

In [ ]:
#read1/2 files are no longer ordered the same and so bwa has issues. switch to bowtie which doesn't care about read ordering
#!mkdir /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/bowtie
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/barcodes_cut/mapping
!bowtie2-build /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/APercula-Reference/GCA_003047355.2_Nemo_v1.1_genomic.fna APPC_ref

In [ ]:
#sort the input read files with bbtools
!for i in $(cat filenames); do repair.sh in1=$i\.R1.fastq.gz in2=$i\.R2.fastq.gz out1=repair/$i.fixed.R1.fq out2=repair/$i.fixed.R2.fq outs=repair/$i.singletons.fq repair; done

In [ ]:
!for i in $(cat filenames); do bowtie2 -x APPC_ref -1 $i\_R1.tagged_filter.fastq.gz -2 $i\_R2.tagged_filter.fastq.gz -S $i.sam; done

In [ ]:
#omg stupid I didn't run cutadapt in paired end mode so it messed up the read names. start in ultraplex out directory and figure out what trimming those files needs
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/

!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*tagged_filter.fastq.gz; do fastqc -t 6 *tagged_filter.fastq.gz  $i; done

In [ ]:
#make sure to check before running this, can't remember where .sai files are made...
#join pairs, convert from sam to bam files in screen -S samsort
#screen -S samsort
for i in $(cat APPC_sequencing/ultraplex_out/fastq_screen/filenames);
do bwa sampe /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/mapping/reference.fasta $i\.read1.sai $i\.read2.sai $i\.R1.tagged_filter.fastq.gz $i\.R2.tagged_filter.fastq.gz | samtools view -bS - >  ~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/$i\.bam; done



In [ ]:
#remove duplicate reads with picard
java -Xms4g -jar /local/home/katrinac/miniconda3/envs/commandline/bin/picard.jar MarkDuplicates I={}-R2.tagged_filter.fastq.gz O={}-RGmd.bam M={}_dup_metrics.txt OPTICAL_DUPLICATE_PIXEL_DISTANCE=2500 TAGGING_POLICY=OpticalOnly &> md.{}.log   

In [ ]:
#January 24, 2022 using fastp to trim adapters because trimmomatic left the barcode in read 2 file
for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/filenames); 
    do fastp -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/$i.R1.fq.gz -I /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/$i.R2.fq.gz -o /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/fastp/$i.R1.fastp.fq.gz -O /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/fastp/$i.R2.fastp.fq.gz; 
    done



In [ ]:
#January 25, there's still barcodes in the beginning of the read 2 files after fastp adapter trimming. I'm going to try ultraplex instead.
cp ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/bcsplit/*.gz ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/
cp ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/bcsplit/*.gz ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/

for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/cap_filenames); 
    do ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/$i-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/$i-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/APPCBarcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/ultraplex_out; 
    done


In [ ]:
#this worked! do this for all capture pools, rename, move this line to the demultiplex notebook, and then redo the fastq, mapping, picard, then angsd and genotype calling
ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP1-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP1-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/Cap1_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/ultraplex_out 